In [9]:
import os
import sys
import socket
import numpy as np
import pandas as pd #also requires xlsxwriter
from at_synapse_detection import dataAccess as da
from at_synapse_detection import antibodyAnalysis as aa

In [3]:
"""Compare two PSD-95 clones.  This is just one way of setting up the 
queries/data locations.  See the 'antibody_analysis' folder for multiple examples 
of how to set up the tool. 

Query Format (dict): 
    preIF : list of strs - name of the channel
    preIF_z : list of ints - number of slices to span. default=[2]
    postIF : list of strs
    postIF_z : list of ints. default=[2]
    punctumSize : number of pixels x/y that a puncta should span. default=2

Returns
-----------------
df : dataframe - contains the results of the SACT 
"""

"Compare two PSD-95 clones.  This is just one way of setting up the \nqueries/data locations.  See the 'antibody_analysis' folder for multiple examples \nof how to set up the tool. \n\nQuery Format (dict): \n    preIF : list of strs - name of the channel\n    preIF_z : list of ints - number of slices to span. default=[2]\n    postIF : list of strs\n    postIF_z : list of ints. default=[2]\n    punctumSize : number of pixels x/y that a puncta should span. default=2\n\nReturns\n-----------------\ndf : dataframe - contains the results of the SACT \n"

In [4]:
# Location of the data 
base_dir = '../example/' 

In [5]:
# Data resolution, in nanometers 
resolution = {'res_xy_nm': 100, 'res_z_nm': 70}

In [6]:
# Threshold value for the probability maps. This value does not usually need to be changed. 
thresh = 0.9

In [7]:
# List the file names.  These must be the same length
target_filenames = ['PSD95m_1st.tif', 'PSD95r_2nd.tif']
target_number_of_slices = [2]
reference_filenames = ['synapsin_1st.tif', 'synapsin_1st.tif']
reference_number_of_slices = [2]


In [8]:
# Create a query for each pair
query_list = []
for n in range(0, len(target_filenames)):
    target_name = target_filenames[n] # The AB we're interested in testing (PSD)
    reference_name = reference_filenames[n] # The previously validated AB (synapsin)

    # Formulate the query
    query = {'preIF': [reference_name], 'preIF_z': reference_number_of_slices, 
             'postIF': [target_name], 'postIF_z': target_number_of_slices,
             'punctumSize': 2}
    query_list.append(query)

# Run the SACT 
measure_list = aa.calculate_measure_lists(query_list, None, base_dir,
                                    thresh, resolution, target_filenames)

# Convert measure object to a dataframe 
project_names = ['Data1', 'Data2']
df = aa.create_df(measure_list, project_names, target_filenames, reference_filenames)
print(df)
df_list = [df]

synapsin_1st.tif
PSD95m_1st.tif
data volume:  10104.640000000001 um3
Computed presynaptic single channel measurements
Computed postsynaptic single channel measurements
starting z ind: 0
starting z ind: 1
starting z ind: 2
starting z ind: 3
starting z ind: 4
starting z ind: 5
starting z ind: 6
starting z ind: 7
starting z ind: 8
starting z ind: 9
synapsin_1st.tif
PSD95r_2nd.tif
data volume:  10104.640000000001 um3
Computed presynaptic single channel measurements
Computed postsynaptic single channel measurements
starting z ind: 0
starting z ind: 1
starting z ind: 2
starting z ind: 3
starting z ind: 4
starting z ind: 5
starting z ind: 6
starting z ind: 7
starting z ind: 8
starting z ind: 9
            Target AB      Conjugate AB  Puncta Density  Puncta Volume  \
Data1  PSD95m_1st.tif  synapsin_1st.tif        0.947980      10.663535   
Data2  PSD95r_2nd.tif  synapsin_1st.tif        1.143237      25.350848   

       Puncta STD  Synapse Density       TSR  Raw Mean    Raw STD  \
Data1  101.6

In [10]:
#Output filename
fn = 'sact_psd_example.xlsx' 
sheet_name = 'sact_psd_example'

In [11]:
# Determine if file exists. This is only an issue when running on windows
if  os.path.isfile(fn): 
    print('A sheet with this name already exists; rename sheet and rerun')
    sys.exit()

# Export the dataframe to excel 
aa.write_dfs_to_excel(df_list, sheet_name, fn)

A sheet with this name already exists; rename sheet and rerun


SystemExit: 

/Users/anish/anaconda3/envs/synapse_analysis/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
